## Module imports

In [1]:
import geopandas as gpd
import numpy as np
import sklearn
gpd.options.io_engine = "pyogrio"
%matplotlib inline

### Import data

In [2]:
train_df = gpd.read_file("./data/train.geojson", index_col=0);
test_df = gpd.read_file("./data/test.geojson", index_col=0);

/Users/foxy/Documents/CS/Machine Learning/.env/lib/python3.11/site-packages/pyogrio/raw.py:194: RuntimeWarning: driver GeoJSON does not support open option INDEX_COL
  result = ogr_read(
/Users/foxy/Documents/CS/Machine Learning/.env/lib/python3.11/site-packages/pyogrio/raw.py:194: RuntimeWarning: driver GeoJSON does not support open option INDEX_COL
  result = ogr_read(


In [3]:
import shapely

# Replace NaN colors with mean 
cols = [col for col in train_df if col.startswith("img")]
test_df[cols] = test_df[cols].fillna(test_df[cols].astype(float).mean())
clean_df = train_df
clean_df[cols] = clean_df[cols].fillna(clean_df[cols].astype(float).mean())

# Delete None change_status
cols = [col for col in clean_df if col.startswith("change_status")]

cols = [col for col in clean_df if col.startswith("date")]
rows = test_df[cols].apply(lambda x : x.isnull() != False ).any(axis = "columns")
test_df.loc[rows, cols] = '1-01-2001'

rows = clean_df[cols].apply(lambda x : x.isnull() != False ).any(axis = "columns")
clean_df.loc[rows, cols] = '1-01-2001'


clean_df.shape

(296146, 45)

## <center> <font color='#0590C0'> Geo data processing <font> <center>

In [87]:

geo = clean_df["geometry"];
geo_test = test_df["geometry"];


#Get area fo training
area_data = geo.apply(lambda x : x.area)
area_data = np.asarray(area_data)
area_data = np.reshape(area_data, (-1, 1))
n_area_data = (area_data - np.mean(area_data, axis=0))/np.std(area_data, axis=0) 

length_data = geo.apply(lambda x : x.length)
length_data = np.asarray(length_data)
length_data = np.reshape(length_data, (-1, 1))
z_length_data = (length_data - np.mean(length_data, axis=0))/np.std(length_data, axis=0) 


# Get area for submission
area_data_test = geo_test.apply(lambda x : x.area)
area_data_test = np.asarray(area_data_test)
area_data_test = np.reshape(area_data_test, (-1, 1))
# min-max normalization
n_area_data_test = (area_data_test - np.mean(area_data, axis=0))/np.std(area_data, axis=0) 

length_data_test = geo_test.apply(lambda x : x.length)
length_data_test = np.asarray(length_data_test)
length_data_test = np.reshape(length_data_test, (-1, 1))
# z normalization
z_length_data_test = (length_data_test - np.mean(length_data, axis=0))/np.std(length_data, axis=0) 
print(n_area_data_test.shape)




(120526, 1)


## <center> <font color='#0590C0'> Geotypes processing <font> <center>

In [5]:
geotypes = set()

geography_types_test = test_df["geography_type"]
geography_types_array_test = np.asarray(test_df["geography_type"])

geography_types = clean_df["geography_type"]
geography_types_array = np.asarray(clean_df["geography_type"])

# List all categories
def count(x) :
    l = x.split(',')
    for m in l :
        if m == 'A' or m =='N' :
            break
        geotypes.add(m)
    return x

count = np.vectorize(count)
count(geography_types_array)

geotypes = list(geotypes)
size = len(geotypes)

geotypes_data = np.ndarray((geography_types_array.shape[0], size))
geotypes_data_test = np.ndarray((geography_types_array_test.shape[0], size))

# Compute for training
for i,t in enumerate(geography_types_array) :
    for m in t.split(',') :
        # N/A value leads to the null vector
        if m == 'A' or m =='N' :
            break
        geotypes_data[i,geotypes.index(m)] = 1

# Compute for submission
for i,t in enumerate(geography_types_array_test) :
    for m in t.split(',') :
        # N/A value leads to the null vector
        if m == 'A' or m =='N' :
            break
        geotypes_data_test[i,geotypes.index(m)] = 1

## <center> <font color='#0590C0'> Urbantypes processing <font> <center>

In [6]:
# Get all types
urbantypes = set()

urban_type_test = test_df["urban_type"]
urban_types_array_test = np.asarray(test_df["urban_type"])

urban_types = clean_df["urban_type"]
urban_types_array = np.asarray(clean_df["urban_type"])

# List all categories
def count(x) :
    l = x.split(',')
    for m in l :
        if m != 'A' and m != 'N' :
            urbantypes.add(m)
    return x

count = np.vectorize(count)
count(urban_types_array)
urbantypes = list(urbantypes)
size = len(urbantypes)

# Compute for training set
urbantypes_data = np.ndarray((urban_types_array.shape[0], size))
for i,t in enumerate(urban_types_array) :
    for m in t.split(',') :
        if m == 'A' or m == 'N' :
            break
        urbantypes_data[i,urbantypes.index(m)] = 1

# Compute for submission set
urbantypes_data_test = np.ndarray((urban_types_array_test.shape[0], size))
for i,t in enumerate(urban_types_array_test) :
    for m in t.split(',') :
        if m == 'A' or m == 'N' :
            break
        urbantypes_data_test[i,urbantypes.index(m)] = 1


[1. 0. 0. 1. 0.]


## <center> <font color='#0590C0'> Status preprocessing <font> <center>

In [7]:
from sklearn.preprocessing import OneHotEncoder

columns = [col for col in clean_df if col.startswith("change_status")]
status = clean_df[columns]
status_test = test_df[columns]

status = np.asarray(status)
status_test = np.asarray(status_test)

# Fit OneHotEncoder
encoder = OneHotEncoder(handle_unknown='ignore')
encoder.fit(status)

# Encode training
status_data = encoder.transform(status).toarray()

# Encode submission
status_data_test = encoder.transform(status_test).toarray()

## <center> <font color='#0590C0'> Date processing <font> <center>

In [8]:
import warnings
import pandas as pd
warnings.filterwarnings("ignore")

columns = [col for col in clean_df if col.startswith("date")]
dates = clean_df[columns]
dates_test = test_df[columns]


for i in range(5) :
    dates[[f'day{i}',f'month{i}',f'year{i}']] = dates[f'date{i}'].str.split('-', expand=True).astype(int)
    dates_test[[f'day{i}',f'month{i}',f'year{i}']] = dates_test[f'date{i}'].str.split('-', expand=True).astype(int)

c_days = [col for col in dates if col.startswith("day")]
c_months = [col for col in dates if col.startswith("month")]
c_year = [col for col in dates if col.startswith("year")]

time = pd.concat([dates[c_days],dates[c_months], dates[c_year]], axis=1)
time = pd.concat([dates_test[c_days],dates_test[c_months], dates_test[c_year]], axis=1)

# Compute date in days
def time(row, j) :
    return row[f'year{j}'] * 365 + row[f'month{j}'] * 30 + row[f'day{j}']  * 1

for i in range(5) :
    dates[f'time{i}'] = dates.apply(time, axis=1, j=i).astype(int)
    dates_test[f'time{i}'] = dates_test.apply(time, axis=1, j=i).astype(int)


In [89]:
# Sort dates beacause they are not
time_array = np.stack(dates[[f'time{i}' for i in range(5)]].astype(int).apply(np.sort, axis=1).to_numpy())
time_array_test = np.stack(dates_test[[f'time{i}' for i in range(5)]].astype(int).apply(np.sort, axis=1).to_numpy())

elapsed = np.ndarray((time_array.shape[0],4))
elapsed_test = np.ndarray((time_array_test.shape[0],4))

# Compute elapsed time
for i in range(4) :
    elapsed[:,i] = time_array[:,i+1] - time_array[:,i]
    elapsed_test[:,i] = time_array_test[:,i+1] - time_array_test[:,i]

# min max normalization
minmax_elapsed = (elapsed - np.min(elapsed, axis=0)) / (np.max(elapsed, axis=0)- np.min(elapsed, axis=0))
minmax_elapsed_test = (elapsed_test - np.min(elapsed, axis=0)) / (np.max(elapsed, axis=0) - np.min(elapsed, axis=0))

## <center> <font color='#0590C0'> Color data processing <font> <center>

In [32]:
# Selects every column starting with "img"
# We could have separated std deviation and mean ?
cols = [col for col in clean_df if "img" in col]

# Build an array from the data
color = np.asarray(clean_df[cols])
mean = np.mean(color, axis=0)
std = np.std(color, axis=0)
ma = np.max(color, axis=0)
mi = np.min(color, axis=0)

# Normalize data
color_test = test_df[cols]
mean_test = np.mean(color_test, axis=0)
std_test = np.std(color_test, axis=0)
ma_test = np.max(color_test, axis=0)
mi_test = np.min(color_test, axis=0)

minimax_color = (color-mi)/(ma-mi)
z_color = (color-mean)/std
z_color_test = (color_test - mean)/std

(296146, 30)


## <center> <font color='#0590C0'> Training <font> <center>

In [90]:
from keras import models, layers, utils
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest, f_classif

# Create training set
labels = ["Demolition", "Road", "Residential", "Commercial", "Industrial", "Mega Projects"]
X = np.concatenate((area_data, geotypes_data, urbantypes_data, status_data, color, length_data), axis=1)

data_labels = clean_df["change_type"]
data_labels = data_labels.apply(lambda x : labels.index(x))

# Vector data set
Y_vector = utils.to_categorical(y,num_classes=6)
X_train, X_test, Y_vector_train, Y_vector_test = train_test_split(X, Y_vector, test_size=0.1, random_state=42)

# Class data set
Y_class = np.asarray(y)

X_train, X_test, Y_class_train, Y_class_test = train_test_split(X, Y_class, test_size=0.1)


In [ ]:
# Train neural network

import tensorflow
from tensorflow import keras
from keras import models, layers
from keras import backend as K

dataset_size = X_train.shape[0]
features = X_train.shape[1]

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

model = models.Sequential()
model.add(layers.Dense(200, activation='relu'))
model.add(layers.Dense(200, activation='relu'))
model.add(layers.Dense(200, activation='relu'))
model.add(layers.Dense(200, activation='relu'))
model.add(layers.Dense(200, activation='relu'))
model.add(layers.Dense(200, activation='relu'))
model.add(layers.Dense(200, activation='relu'))
model.add(layers.Dense(200, activation='relu'))
model.add(layers.Dense(200, activation='relu'))
model.add(layers.Dense(200, activation='relu'))
model.add(layers.Dense(200, activation='relu'))


model.add(layers.Dense(6, activation='softmax'))
model.build((dataset_size,features))

model.compile(loss = 'categorical_crossentropy' , optimizer='sgd', metrics=['accuracy', f1_m])
model.fit(X_train, Y_vector_train,batch_size=30, validation_split=0.2, epochs=5)


## <center> <font color='#0590C0'> Training RandomForest <font> <center>

In [104]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_estimators=200, max_depth=40)
clf.fit(X_train, Y_class_train)
# Compute training accuracy to check overfitting
print(clf.score(X_train[:5000],Y_class_train[:5000]))
# Compute validation accuracy
print(clf.score(X_test,Y_class_test))
y_pred = clf.predict(X_test)
# Compute f1 score
sklearn.metrics.f1_score(y_pred, Y_class_test,  average='micro')

1.0
0.7238899206483201


0.7238899206483201

## <center> <font color='#0590C0'> Submission file <font> <center>

In [105]:
X_submission = np.concatenate((area_data_test, geotypes_data_test, urbantypes_data_test, status_data_test, color_test, length_data_test), axis=1)
Y_submission = clf.predict(X_submission)

In [106]:
s = pd.DataFrame(Y_submission, columns=['change_type'])
s.index.name = "Id"
s.to_csv('test7.csv')